Load in the Dataset

In [1]:
import kagglehub
import os
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("lonnieqin/englishspanish-translation-dataset")

print("Path to dataset files:", path)

# Check what files are inside that directory
print(os.listdir(path))   # 👈 this shows filenames, like ['english-spanish.csv']

# Pick the right CSV file
csv_path = os.path.join(path, "data.csv")  # adjust name after seeing output
data = pd.read_csv(csv_path)

c:\ML\Translator(Transformer)\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\metik\.cache\kagglehub\datasets\lonnieqin\englishspanish-translation-dataset\versions\1
['data.csv']


Create english and spanish tokens

In [2]:
import re

TOKEN_RE = re.compile(r"[A-Za-zÁÉÍÓÚÜÑáéíóúüñ]+(?:'[A-Za-z]+)?|[.,!?;:]", flags=0)

def fast_tok(s: str):
    return TOKEN_RE.findall(s)

#English and spanish tokens
english_tokens = [fast_tok(s) for s in data['english']]
spanish_tokens = [fast_tok(s) for s in data['spanish']]

Assign an id to each unique word (English)

In [3]:
index_dict_english = {}

#Length of English dict is 14943
for sen_index in range(len(english_tokens)):
  for word in english_tokens[sen_index]:
    if word not in index_dict_english:
      index_dict_english[word] = len(index_dict_english)

index_dict_english["<EOS>"] = len(index_dict_english)
PAD_EN = len(index_dict_english); index_dict_english["<PAD>"] = PAD_EN

index_dict_english["<UNK>"] = len(index_dict_english)
UNK_EN = index_dict_english["<UNK>"]

Assign an id to each unique word (Spanish)

In [4]:
index_dict_spanish = {}

#Length of Spanish dict is 28777
for sen_index in range(len(spanish_tokens)):
  for word in spanish_tokens[sen_index]:
    if word not in index_dict_spanish:
      index_dict_spanish[word] = len(index_dict_spanish)

index_dict_spanish["<EOS>"] = len(index_dict_spanish)
index_dict_spanish["<BOS>"] = len(index_dict_spanish)
PAD_ES = len(index_dict_spanish); index_dict_spanish["<PAD>"] = PAD_ES

Create random initial word embeddings for English and Spanish words

In [5]:
import torch
import torch.nn as nn

vocab_size_english = len(index_dict_english)
vocab_size_spanish = len(index_dict_spanish)
embedding_dim = 256

english_emb = nn.Embedding(vocab_size_english, embedding_dim, padding_idx=PAD_EN)
spanish_emb = nn.Embedding(vocab_size_spanish, embedding_dim, padding_idx=PAD_ES)

Turning words into sentences to their corresponding ids

In [6]:
english_ids = []
spanish_ids = []

for sen in english_tokens:
  ids = [index_dict_english.get(word, UNK_EN) for word in sen]
  ids.append(index_dict_english["<EOS>"])
  english_ids.append(torch.tensor(ids, dtype=torch.long))

for sen in spanish_tokens:
  ids = [index_dict_spanish["<BOS>"]] + [index_dict_spanish[word] for word in sen] + [index_dict_spanish["<EOS>"]]
  spanish_ids.append(torch.tensor(ids, dtype=torch.long))

Shuffle data

In [7]:
import numpy as np

indices = np.random.permutation(len(english_ids))

shuffled_english_ids = [english_ids[i] for i in indices]
shuffled_spanish_ids = [spanish_ids[i] for i in indices]

Custom Dataset and Train/Test spilt

In [8]:
from torch.utils.data import Dataset

class TranslationDataset(Dataset):
    def __init__(self, src_ids, trg_ids):
        self.src = src_ids
        self.trg = trg_ids

    def __len__(self):
        return len(self.src)

    def __getitem__(self, idx):
        return self.src[idx], self.trg[idx]

size = data.shape[0] #118964
train_size = int(size * 0.8)
test_size = int(size * 0.1)
val_size = size - train_size - test_size

train_data = TranslationDataset(
    shuffled_english_ids[:train_size],
    shuffled_spanish_ids[:train_size]
)

test_data = TranslationDataset(
    shuffled_english_ids[train_size:-val_size],
    shuffled_spanish_ids[train_size:-val_size]
)

val_data = TranslationDataset(
    shuffled_english_ids[-val_size:],
    shuffled_spanish_ids[-val_size:]
)

Custom collate_fn and initializing Dataloaders

In [9]:
def collate_fn(batch):
    ids_en = [item[0] for item in batch]
    ids_es = [item[1] for item in batch]
    return ids_en, ids_es

from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_data,
    batch_size=256,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    collate_fn=collate_fn
)

test_loader = DataLoader(
    test_data,
    batch_size=256,
    shuffle=False,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    val_data,
    batch_size=256,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    collate_fn=collate_fn
)

Training the Seq2Seq model

In [ ]:
import os
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
import lightning as L
from models.Seq2Seq import Seq2Seq
from models.EncoderTrans import EncoderTrans
from models.DecoderTrans import DecoderTrans

model = Seq2Seq(
    encoder=EncoderTrans(english_emb, PAD_EN),
    decoder=DecoderTrans(spanish_emb, PAD_ES),
    pad_en=PAD_EN,
    pad_es=PAD_ES,
    eos_es=index_dict_spanish["<EOS>"],
    bos_es=index_dict_spanish["<BOS>"],
    lr=3e-4
)

logger = TensorBoardLogger(save_dir="logs", name="seq2seq_en_es")

checkpoint_cb = ModelCheckpoint(
    dirpath="checkpoints",
    filename="epoch{epoch}-valloss{val_loss:.3f}",
    save_last=True,
    save_top_k=1,
    monitor="val_loss",
    mode="min"
)

trainer = L.Trainer(
    max_epochs=15,
    accelerator="gpu", 
    devices="auto",         
    callbacks=[checkpoint_cb],
    logger=logger,
    log_every_n_steps=10,
    precision="16-mixed",
    num_sanity_val_steps=0,
    gradient_clip_val=1.0
)

ckpt = "checkpoints/last.ckpt"
trainer.fit(
    model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
    ckpt_path=ckpt if os.path.exists(ckpt) else None
)


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
c:\ML\Translator(Transformer)\.venv\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:881: Checkpoint directory C:\ML\Translator(Transformer)\checkpoints exists and is not empty.
Restoring states from the checkpoint path at checkpoints/last.ckpt
c:\ML\Translator(Transformer)\.venv\Lib\site-packages\lightning\fabric\utilities\cloud_io.py:73: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer 

Epoch 14: 100%|██████████| 372/372 [00:48<00:00,  7.65it/s, v_num=1, train_loss_step=0.981, train_tok_acc_step=0.749, val_loss=2.180, val_tok_acc=0.626, train_loss_epoch=0.950, train_tok_acc_epoch=0.759]

`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: 100%|██████████| 372/372 [00:48<00:00,  7.65it/s, v_num=1, train_loss_step=0.981, train_tok_acc_step=0.749, val_loss=2.180, val_tok_acc=0.626, train_loss_epoch=0.950, train_tok_acc_epoch=0.759]


Helper methods

In [19]:
id2en = {i:w for w,i in index_dict_english.items()}
id2es = {i:w for w,i in index_dict_spanish.items()}

en2id = index_dict_english
es2id = index_dict_spanish

def ids_to_text(ids, id2word, pad_id=None, eos_id=None, bos_id=None):
  words = []
  for i in ids:
    i = int(i)
    if pad_id is not None and i == pad_id:
      continue
    if bos_id is not None and i == bos_id:
      continue
    if eos_id is not None and i == eos_id:
      break  # hide EOS in printing
    words.append(id2word.get(i, "<UNK>"))
  return " ".join(words)

def translate(sentence: str, model, en2id, id2es, PAD_ES, EOS_ES, BOS_ES, UNK_EN, use_beam_search=True, beam_size = 5, max_len=40, len_norm_alpha=0.3):
  model.eval()
  device = next(model.parameters()).device

  tokens = sentence.strip().split()
  src_ids = torch.tensor(
    [en2id.get(t, UNK_EN) for t in tokens] + [en2id["<EOS>"]],
    dtype=torch.long,
    device=device
  )

  if use_beam_search:
    pred_ids = model.beam_search(src_ids, beam_size, max_len, len_norm_alpha)
  else:
    pred_ids = model.greedy_decode(src_ids, max_len=max_len)

  pred_ids = pred_ids.reshape(-1)
  return ids_to_text(pred_ids, id2es, pad_id=PAD_ES, eos_id=EOS_ES, bos_id=BOS_ES)

Load weights from ./checkpoints

In [12]:
# import torch
# import lightning as L

# from models.Seq2Seq import Seq2Seq
# from models.EncoderTrans import EncoderTrans
# from models.DecoderTrans import DecoderTrans

# encoder = EncoderTrans(english_emb, PAD_EN)
# decoder = DecoderTrans(spanish_emb, PAD_ES)

# # ckpt_path = r"checkpoints/epochepoch=12-vallossval_loss=2.185.ckpt"
# ckpt_path = r"checkpoints/last.ckpt"   

# model = Seq2Seq.load_from_checkpoint(
#     ckpt_path,
#     encoder=encoder,
#     decoder=decoder,
#     pad_en=PAD_EN,
#     pad_es=PAD_ES,
#     eos_es=index_dict_spanish["<EOS>"],
#     bos_es=index_dict_spanish["<BOS>"],
#     lr=1e-3
# )

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

Testing

In [20]:
import random

model.eval()

for _ in range(5):
  idx = random.randrange(len(val_data))
  en_ids, es_ids = val_data[idx]

  # Turn EN ids -> sentence text (so translate() can tokenize consistently)
  en_text = ids_to_text(en_ids, id2en, pad_id=PAD_EN, eos_id=index_dict_english["<EOS>"])
  gt_text = ids_to_text(es_ids, id2es, pad_id=PAD_ES, eos_id=index_dict_spanish["<EOS>"], bos_id=index_dict_spanish["<BOS>"])

  greedy_text = translate(
    en_text, model, en2id, id2es, PAD_ES, index_dict_spanish["<EOS>"], index_dict_spanish["<BOS>"], UNK_EN,
    use_beam_search=False, max_len=50
  )

  beam_text = translate(
    en_text, model, en2id, id2es, PAD_ES, index_dict_spanish["<EOS>"], index_dict_spanish["<BOS>"], UNK_EN,
    use_beam_search=True, beam_size=3
  )

  print("EN:", en_text)
  print("GT:", gt_text)
  print("GR:", greedy_text)
  print("BM:", beam_text)
  print("-" * 60)

EN: Tom's French is pretty good .
GT: El francés de Tom es bastante bueno .
GR: La verdad de Tom es bastante bueno .
BM: La verdad de Tom es bastante bueno .
------------------------------------------------------------
EN: He tried to comfort her , but she kept crying .
GT: Intentó consolarla , pero ella no paraba de llorar .
GR: Él intentó consolarlo , pero siguió ella intentó consolarlo .
BM: Él trató de llorar .
------------------------------------------------------------
EN: Did he fail again ?
GT: Fracasó de nuevo ?
GR: Él se volvió otra vez ?
BM: Él volvió otra vez ?
------------------------------------------------------------
EN: I made some modifications .
GT: Hice algunas modificaciones .
GR: Hice poco algunos ensayos .
BM: Hice algunas .
------------------------------------------------------------
EN: My house has two bedrooms .
GT: Mi casa tiene dos dormitorios .
GR: Mi casa tiene dos habitaciones .
BM: Mi casa .
------------------------------------------------------------


Testing

In [22]:
trainer.validate(model, dataloaders=test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation DataLoader 0: 100%|██████████| 47/47 [00:01<00:00, 27.74it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        val_loss             2.16614031791687
       val_tok_acc           0.628110408782959
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 2.16614031791687, 'val_tok_acc': 0.628110408782959}]

Playing

In [29]:
sentence = "I like eating apples ."
beam_text = translate(
        sentence, model, en2id, id2es, PAD_ES, index_dict_spanish["<EOS>"], index_dict_spanish["<BOS>"], UNK_EN,
        use_beam_search=True, beam_size=3
    )
print(beam_text)

Me gustan las manzanas .
